# Język Python - Wykład 7

Python3: "A general goal is to reduce feature duplication by removing old ways of doing things" (PEP 3100)

Różnice Python 2.x i 3.x

 - print vs print()
 - UTF i kodowanie
 - zip vs zip_longest
 - metaclasses
 - nowe i stare klasy
 - slowa kluczowe, True i False
 - biblioteki

In [ ]:
s = "ala ma kota"

In [ ]:
s[0]

In [ ]:
s[0] = 'b'

Python 3.x
  - Unicode domyślnie (w tym ASCII)
  - osobny typ dla danych binarnych
  
Python 2.x
  - ASCII domyślnie
  - dane binarnie domyślnie
  - osobny typ dla Unicode (spoza ASCII)

Mark Lutz - Learning Python (4th edition)

In [ ]:
print("żółw")

In [ ]:
# o typach danych

help(str)  # from 2.x str
help(bytes)  # from 2.x unicode
help(bytearray)

In [ ]:
import sys
print(sys.getdefaultencoding())

In [ ]:
ord('a')

In [ ]:
hex(97)

In [ ]:
chr(97)

In [ ]:
chr(0xC4)

In [ ]:
import codecs
import tempfile
import os.path

filename = os.path.join( tempfile.gettempdir() , "L6" )
with codecs.open(filename, mode='w', encoding='iso-8859-2') as f:
    f.write(u'żółw')
    
with codecs.open(filename, encoding='iso-8859-2') as f:
    string = f.readline()
    print(string, repr(string))

## print

Arguments (PEP 3105 -- Make print a function):
  - print is the only application-level functionality that has a statement (syntax!) dedicated to it. Syntax should cover only necessary items. 
  - print() can be easily replaced by more sophisticated functions, while print doesn't (>> syntax!)
  - in print - not easy to change separator from space to other character

In [ ]:
#%%python3
print('Hello world', 'here I come', sep="")

In [ ]:
#%%python2
print 'Hello world','here I come'

In [ ]:
#%%python3
def printnew(*args, **kwargs):
    __builtins__.print('AGH')
    return __builtins__.print(*args, **kwargs)
print = printnew
print('Hello world')
print = __builtins__.print

In [ ]:
#%%python2
def printnew(*args, **kwargs):
    __builtins__.print('AGH')
    return __builtins__.print(*args, **kwargs)
print = printnew
print('Hello world')
print = __builtins__.print

### Chevron print

Chevron (pl. szewron) - naszywka na rękawie lub naramienniku munduru, w kształcie prostej lub odwróconej litery "V"
https://pl.wikipedia.org/wiki/Szewron_(naszywka) https://en.wikipedia.org/wiki/Chevron_(insignia)

In [ ]:
#%%python3
import sys
print('Error!', file=sys.stderr)
print('Not an error', file=sys.stdout)
print('Error!', file=sys.stderr)

In [ ]:
%%python2
print >> sys.stderr, 'Error!'
print >> sys.stdout, 'Not an error'
print >> sys.stderr, 'Error!'

## map, filter, reduce

### Views And Iterators Instead Of Lists

In [ ]:
##python2
isPrime=lambda x: all(x % i != 0 for i in range(int(x**0.5)+1)[2:])
import numpy as np
np.array(filter(isPrime,[10,20,30,13,7]))

In [ ]:
##python3
isPrime=lambda x: all(x % i != 0 for i in range(int(x**0.5)+1)[2:])
import numpy as np
np.array(list(filter(isPrime,[10,20,30,13,7])))

### map

In [ ]:
##python2
def p(*args):
   print [str(x) for x in args]
list(map( p, [1,1,1,1],[1] ))

In [ ]:
##python3
def p(*args):
   print([str(x) for x in args])
list(map( p, [1,1,1,1],[1] ))

In [ ]:
##python2
def transposed(matrix):
    """Return transposed matrix (list of lists).

    This function can handle non-square matrices.
    In this case it fills shorter list with None.

    >>> transposed( [[1,2,3], [3,4]] )
    [[1, 3], [2, 4], [3, None]]
    """
    return map(lambda *row: list(row), *matrix)

In [ ]:
##python3
def transposed(matrix):
    """Return transposed matrix (list of lists).

    This function can handle non-square matrices.
    In this case it fills shorter list with None.

    >>> transposed( [[1,2,3], [3,4]] )
    [[1, 3], [2, 4], [3, None]]
    """
    return list(map(list, zip_longest(*matrix)))

### reduce

Python 3000 FAQ
http://www.artima.com/weblogs/viewpost.jsp?thread=211200

Q. If you're killing reduce(), why are you keeping map() and filter()?

A. I'm not killing reduce() because I hate functional programming; I'm killing it because almost all code using reduce() is less readable than the same thing written out using a for loop and an accumulator variable. On the other hand, map() and filter() are often useful and when used with a pre-existing function (e.g. a built-in) they are clearer than a list comprehension or generator expression. (Don't use these with a lambda though; then a list comprehension is clearer and faster.)

In [ ]:
#python2
def mul(x,y): return x/2.0+1/x
reduce(mul, range(1, 11))

In [ ]:
#python3
import functools
def mul(x,y): return x/2.0+1/x
functools.reduce(mul, range(1, 11))

In [ ]:
result = 1
for x in range(1,11):
    result = mul(result, x)
result

### range

In [ ]:
##python2
range(10)
xrange(10)

In [ ]:
##python3
range(10)
xrange(10)

### zip

In [ ]:
##python2
A = [1,2,3,4]
B = ['a','b','c']
zip(A,B)

In [ ]:
##python3
A = [1,2,3]
B = ['a','b','c']
zip(A,B)

## map, filter, reduce

In [ ]:
#python2
1 < 'a'
sorted([3,1,2,'b','a','c'])

In [ ]:
#python3
1 < 'a'
sorted([3,1,2,'b','a','c'])

## integers
long renamed to int

In [ ]:
##python2
int("100")
long("100")

In [ ]:
##python3
int("100")
long("100")

https://www.python.org/dev/peps/pep-0238/
The current division (/) operator has an ambiguous meaning for
    numerical arguments: it returns the floor of the mathematical
    result of division if the arguments are ints or longs, but it
    returns a reasonable approximation of the division result if the
    arguments are floats or complex.  This makes expressions expecting
    float or complex results error-prone when integers are not
    expected but possible as inputs.
    
    The problem is unique to dynamically typed languages: in a
    statically typed language like C, the inputs, typically function
    arguments, would be declared as double or float, and when a call
    passes an integer argument, it is converted to double or float at
    the time of the call.
    
    The correct work-around is subtle: casting an argument to float()
    is wrong if it could be a complex number; adding 0.0 to an
    argument doesn't preserve the sign of the argument if it was minus
    zero.  The only solution without either downside is multiplying an
    argument (typically the first) by 1.0.  This leaves the value and
    sign unchanged for float and complex, and turns int and long into
    a float with the corresponding value.

In [ ]:
#python2
1/2

In [ ]:
#python3
1/2, 1//2

In [ ]:
import math
math.atan2(0.0,-0.0)

In [ ]:
math.atan2(-0.0,-0.0)

In [ ]:
0.0 == -0.0

## Wyjatki

One of Python's guiding maxims is "there should be one -- and preferably only one -- obvious way to do it" [1] . Python 2.x's raise statement violates this principle, permitting multiple ways of expressing the same thought

In [ ]:
##python2
class MyEx(Exception):
    x = 0
    y = 0
    
einst = MyEx()
einst.x = 1
einst.y = 1

In [ ]:
#python2
#raise_stmt: 'raise' [test [',' test [',' test]]]
try:
    raise einst
#    raise MyEx, einst
except MyEx, e:
    print e.x, e.y

In [ ]:
#python3
#raise_stmt: 'raise' [test]
try:
    raise einst
except MyEx as e:
    print(e.x, e.y)

In [ ]:
#python2
try:
    1/0
except Exception:
    raise TypeError

In [ ]:
class FooException(Exception):
    pass

class BarException(Exception):
    pass

class foo(object):
    def d(self):
       self.e()

    def e(self):
       self.f()

    def f(self):
       raise FooException('Problem')

class bar(object):
    def a(self):
       self.c()

    def b(self):
       self.c()

    def c(self):
        try:
            f = foo()
            f.d()
        except FooException as e:
            raise BarException(e)

bar().a()

## List comprehension

In [ ]:
x = 'before'
a = [x for x in [1, 2, 3]]
print(x)

## Rozpakowywanie krotek
PEP 3113 -- Removal of Tuple Parameter Unpacking
Unfortunately this feature of Python's rich function signature abilities, while handy in some situations, causes more issues than they are worth. 

In [ ]:
def middle( (a,b), (c,d) ):
    return 0.5*(a+c),0.5*(b+d)
middle( (1,0), (13,-4))

In [ ]:
def middle( p1, p2 ):
    a,b = p1
    c,d = p2
    return 0.5*(a+c),0.5*(b+d)
middle( (1,0), (13,-4))